# Watersheds for Supervised Classification

In this notebook we present a few results on using watersheds for supervised classification. We use a variant of the MSF-watershed as cited in [1](https://hal-upec-upem.archives-ouvertes.fr/hal-00622410/document) for supervised classification.


All the relevant implementations are in "utils.py". 

Links:

[1] https://hal-upec-upem.archives-ouvertes.fr/hal-00622410/document


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time

from utils import *

%matplotlib inline


from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC


# Visualizing the boundaries

We start with trying to understand the how the boundaries of using watershed as classifier behave? Note that MSF-watershed requires both the train and test data to be presented before training. However, as discussed in the accompanying article, one can use 1-Nearest Neighbor classifier to classify new points and this would result in equivalent labelling! (Thanks to PRIM's MST algorithm)

We use this techinique to visualize the boundaries!!

In [ ]:
from sklearn.datasets import make_blobs, make_circles, make_moons
from sklearn.neighbors import kneighbors_graph

np.random.seed(10)

size_data = 1000
percent_seeds = 0.1
X, y = make_blobs(size_data, n_features=2, centers=[[1,1], [5,5]], cluster_std=1.0)
graph = kneighbors_graph(X, n_neighbors=20, mode='distance')
seeds = np.zeros(size_data, dtype=np.int32)

for label in np.unique(y):
    ind_label = np.where(y==label)[0]
    ind = np.random.choice(ind_label, size=int(percent_seeds*(ind_label.shape[0])))
    seeds[ind] = label+1
    
y_watershed = watershed_supervised(graph, seeds)

bx, by = plot_boundary(X, y_watershed, resolution=400, n_neighbors=5)
xmin, xmax, ymin, ymax = np.min(X[:,0]), np.max(X[:,0]), np.min(X[:,1]), np.max(X[:,1])
color = np.array([x for x in 'bgrcmykw']*10)

plt.figure(figsize=(20,7))
plt.subplot(1,3,1)
plt.axis([xmin, xmax, ymin, ymax])
ind = np.where(y_watershed == 1)
plt.scatter(X[ind,0], X[ind,1], marker='+', c='0.5')
ind = np.where(y_watershed == 2)
plt.scatter(X[ind,0], X[ind,1], marker='o', c='0.8')
plt.scatter(bx, by, s=1,c='k')
plt.savefig("./img/example1.eps", dpi=400, cmap=plt.cm.gray)


size_data = 1000
percent_seeds = 0.1
X, y = make_circles(n_samples=size_data, noise=0.03, factor=0.7)
graph = kneighbors_graph(X, n_neighbors=20, mode='distance')
seeds = np.zeros(size_data, dtype=np.int32)

for label in np.unique(y):
    ind_label = np.where(y==label)[0]
    ind = np.random.choice(ind_label, size=int(percent_seeds*(ind_label.shape[0])))
    seeds[ind] = label+1
    
y_watershed = watershed_supervised(graph, seeds)

bx, by = plot_boundary(X, y_watershed, resolution=400, n_neighbors=5)
xmin, xmax, ymin, ymax = np.min(X[:,0]), np.max(X[:,0]), np.min(X[:,1]), np.max(X[:,1])
color = np.array([x for x in 'bgrcmykw']*10)

plt.subplot(1,3,2)
plt.axis([xmin, xmax, ymin, ymax])
ind = np.where(y_watershed == 1)
plt.scatter(X[ind,0], X[ind,1], marker='+', c='0.5')
ind = np.where(y_watershed == 2)
plt.scatter(X[ind,0], X[ind,1], marker='o', c='0.8')
plt.scatter(bx, by, s=1,c='k')
plt.savefig("./img/example2.eps", dpi=400, cmap=plt.cm.gray)


size_data = 1000
percent_seeds = 0.1
X, y = make_moons(n_samples=size_data, noise=0.1)
graph = kneighbors_graph(X, n_neighbors=20, mode='distance')
seeds = np.zeros(size_data, dtype=np.int32)

for label in np.unique(y):
    ind_label = np.where(y==label)[0]
    ind = np.random.choice(ind_label, size=int(percent_seeds*(ind_label.shape[0])))
    seeds[ind] = label+1
    
y_watershed = watershed_supervised(graph, seeds)

bx, by = plot_boundary(X, y_watershed, resolution=400, n_neighbors=5)
xmin, xmax, ymin, ymax = np.min(X[:,0]), np.max(X[:,0]), np.min(X[:,1]), np.max(X[:,1])
color = np.array([x for x in 'bgrcmykw']*10)

plt.subplot(1,3,3)
plt.axis([xmin, xmax, ymin, ymax])
ind = np.where(y_watershed == 1)
plt.scatter(X[ind,0], X[ind,1], marker='+', c='0.5')
ind = np.where(y_watershed == 2)
plt.scatter(X[ind,0], X[ind,1], marker='o', c='0.8')
plt.scatter(bx, by, s=1,c='k')
plt.savefig("./img/example3.eps", dpi=400, cmap=plt.cm.gray)



# Experiments on High Dimensional Data

The following algorithms in utils.py are used

1. IFT(graph, seeds, alg='SP-SUM')
2. RW(graph, seeds, beta=5., eps=1e-6)
3. powerWatershed_multipleLabels(graph, seeds, bucketing='kmeans', eps=1e-2, k=3, beta=5., eps_weight=1e-6)
4. ensemble_watershed(X, graph, seeds, number_estimators=10, num_features_select=20, percentage_seed_select = 80, gt=None) 
5. watershed_supervised(graph, seeds)
6. SVM (From Sklearn)
7. 1-NearestNeighbor (From Sklearn)
8. RandomForest (From Sklearn)

The SSL datasets are used. The function "get_data(name)" returns the data. 

Possible Parameters:
* IFT : alg='SP-SUM'
* RW : beta in [1,3,5] and eps=[1e-6]
* powerWatershed_multipleLabels : k in [3,5,10], beta in [1,3,5]
* ensemble_watershed: number_estimators=100,  num_features_select in [20,30,50]%
* SVM : C in [0.1, 1., 3.]
* 1-NN (Nothing)
* RandomForest : num_features_select in [20,30,50]%

20% of the data is used for training!

In [ ]:
for dataset in ['SSL1', 'SSL2', 'SSL3','SSL4', 'SSL5','SSL6', 'SSL7']:
    X, y, graph = get_data(dataset)
    y = y.ravel()
    seeds = np.zeros(X.shape[0], dtype=np.int32)
    for label in np.unique(y):
        tmp = np.where(y==label)[0]
        indSelect = np.random.choice(tmp, int(0.01*percent_train*tmp.shape[0]), replace=False)
        seeds[indSelect] = label
        assert label > 0, "GT Label is 0 for some dataset. Please check..."
    
    y_PW = powerWatershed_multipleLabels(graph, seeds, bucketing='kmeans', eps=1e-2, k=1, beta=3, eps_weight=1e-6)
    print(np.mean(y_PW[seeds==0]==y[seeds==0]))
    
    y_RW = RW(graph, seeds, beta=3, eps=1e-6)
    print(np.mean(y_RW[seeds==0]==y[seeds==0]))

In [60]:
percent_train = 20
count = 0
for iIter in range(20):
    for dataset in ['SSL1', 'SSL2', 'SSL3','SSL4', 'SSL5','SSL6', 'SSL7']:
        X, y, graph = get_data(dataset)
        y = y.ravel()
        seeds = np.zeros(X.shape[0], dtype=np.int32)
        for label in np.unique(y):
            tmp = np.where(y==label)[0]
            indSelect = np.random.choice(tmp, int(0.01*percent_train*tmp.shape[0]), replace=False)
            seeds[indSelect] = label
            assert label > 0, "GT Label is 0 for some dataset. Please check..."
        
        # Watershed Supervised
        fname = "./results/results_watershed_supervised.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")
        tic = time.time()
        y_watershed = watershed_supervised(graph,seeds)
        toc = time.time()
        acc_watershed = np.mean(y_watershed[seeds==0]==y[seeds==0])
        with open(fname,'a') as f:
            f.write(str(count)+","+"watershed"+","+dataset+","+"NA"+","+str(acc_watershed)+","+str(toc-tic))
            f.write("\n")
        count += 1
        print("Watershed for {} done...".format(dataset))

        # IFT-SUM
        fname = "./results/results_IFT_SUM.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")        
        tic = time.time()
        y_IFT = IFT(graph, seeds, alg='SP-SUM')
        toc = time.time()
        acc_IFT = np.mean(y_IFT[seeds==0]==y[seeds==0])
        with open(fname,'a') as f:
            f.write(str(count)+","+"IFT-SUM"+","+dataset+","+"NA"+","+str(acc_IFT)+","+str(toc-tic))
            f.write("\n")
        count += 1
        print("IFT-SUM for {} done...".format(dataset))

        # RW
        fname = "./results/results_RW.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n") 
        for beta in [1., 3., 5.]:
            tic = time.time()
            y_RW = RW(graph, seeds, beta=beta, eps=1e-6)
            toc = time.time()
            acc_RW = np.mean(y_RW[seeds==0]==y[seeds==0])
            with open(fname,'a') as f:
                f.write(str(count)+","+"RW"+","+dataset+","+"beta="+str(beta)+","+str(acc_RW)+","+str(toc-tic))
                f.write("\n")
            count += 1
            print("RW for {} with param beta={} done...".format(dataset, beta))


        # Power Watershed
        fname = "./results/results_PW.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n") 
        for beta in [1., 3., 5.]:
            for k in [1,3,5,10]:
                tic = time.time()
                y_PW = powerWatershed_multipleLabels(graph, seeds, bucketing='kmeans', eps=1e-2, k=k, beta=beta, eps_weight=1e-6)
                toc = time.time()
                acc_PW = np.mean(y_PW[seeds==0]==y[seeds==0])
                with open(fname, 'a') as f:
                    f.write(str(count)+","+"PW"+","+dataset+","+"beta="+str(beta)+":k="+str(k)+","+str(acc_PW)+","+str(toc-tic))
                    f.write("\n")
                count += 1
                print("PW for {} with param beta={} k={} done...".format(dataset, beta, k))


        # SVM
        fname = "./results/results_SVM.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")         
        for C in [0.1, 1., 3.]:
            clf = SVC(C=C, gamma='auto')
            tic = time.time()
            clf.fit(X[seeds>0],y[seeds>0])
            y_SVM = clf.predict(X)
            toc = time.time()
            acc_SVM = np.mean(y_SVM[seeds==0]==y[seeds==0])
            with open(fname, 'a') as f:            
                f.write(str(count)+","+"SVM"+","+dataset+","+"C="+str(C)+","+str(acc_SVM)+","+str(toc-tic))
                f.write("\n")
            count += 1
            print("SVM for {} with param C={} done...".format(dataset, C))

        # 1-NN     
        fname = "./results/results_1NN.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")    
        clf = KNN(n_neighbors=1)
        tic = time.time()
        clf.fit(X[seeds>0],y[seeds>0])
        y_1NN = clf.predict(X)
        toc = time.time()
        acc_1NN = np.mean(y_1NN[seeds==0]==y[seeds==0])
        with open(fname, 'a') as f:          
            f.write(str(count)+","+"1NN"+","+dataset+","+"NA"+","+str(acc_1NN)+","+str(toc-tic))
            f.write("\n")
        count += 1
        print("1NN for {} done...".format(dataset))


        # Random Forest
        fname = "./results/results_RFC.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")            
        for depth in range(3,21):
            clf = RFC(n_estimators=100, max_depth=depth)
            tic = time.time()
            clf.fit(X[seeds>0],y[seeds>0])
            y_RFC = clf.predict(X)
            toc = time.time()
            acc_RFC = np.mean(y_RFC[seeds==0]==y[seeds==0])
            with open(fname, 'a') as f:
                f.write(str(count)+","+"RFC"+","+dataset+","+"max_depth="+str(depth)+","+str(acc_RFC)+","+str(toc-tic))
                f.write("\n")
            print("RFC for {}  done...".format(dataset))
            count += 1

        # Ensemble Watersheds
        fname = "./results/results_EnsembleWatersheds.csv"
        if iIter == 0 and dataset=='SSL1':
            with open(fname, 'w') as f:
                f.write("id,method,dataset,paramters,accuracy,time\n")            
        for num_features in [20,30,50]:
            tic = time.time()
            y_ensembleWatershed, _ = ensemble_watershed(X, graph, seeds, number_estimators=100, num_features_select=num_features, percentage_seed_select = 80, gt=None) 
            toc = time.time()
            acc_ensembleWatershed = np.mean(y_ensembleWatershed[seeds==0]==y[seeds==0])
            with open(fname, 'a') as f:
                f.write(str(count)+","+"Ensemblewatershed"+","+dataset+","+"num_features="+str(num_features)+","+str(acc_ensembleWatershed)+","+str(toc-tic))
                f.write("\n")
            count += 1
            print("Ensemble Watersheds for {} with param num_features={} done...".format(dataset, num_features))

        

Watershed for SSL1 done...
priority queue is empty
IFT-SUM for SSL1 done...
RW for SSL1 with param beta=1.0 done...
RW for SSL1 with param beta=3.0 done...
RW for SSL1 with param beta=5.0 done...
PW for SSL1 with param beta=1.0 k=1 done...
PW for SSL1 with param beta=1.0 k=3 done...
PW for SSL1 with param beta=1.0 k=5 done...
PW for SSL1 with param beta=1.0 k=10 done...
PW for SSL1 with param beta=3.0 k=1 done...
PW for SSL1 with param beta=3.0 k=3 done...
PW for SSL1 with param beta=3.0 k=5 done...
PW for SSL1 with param beta=3.0 k=10 done...
PW for SSL1 with param beta=5.0 k=1 done...
PW for SSL1 with param beta=5.0 k=3 done...
PW for SSL1 with param beta=5.0 k=5 done...
PW for SSL1 with param beta=5.0 k=10 done...
SVM for SSL1 with param C=0.1 done...
SVM for SSL1 with param C=1.0 done...
SVM for SSL1 with param C=3.0 done...
1NN for SSL1 done...
RFC for SSL1  done...
RFC for SSL1  done...
RFC for SSL1  done...
RFC for SSL1  done...
RFC for SSL1  done...
RFC for SSL1  done...
RFC fo

/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))
/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))


PW for SSL5 with param beta=3.0 k=3 done...


/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))
/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))


PW for SSL5 with param beta=3.0 k=5 done...


...

/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))
/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))


PW for SSL7 with param beta=5.0 k=5 done...


/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))
/home/csa/Documents/Aditya/morphMedian/utils.py:686: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  clf.fit(w.reshape((-1, 1)))


PW for SSL7 with param beta=5.0 k=10 done...
SVM for SSL7 with param C=0.1 done...
SVM for SSL7 with param C=1.0 done...
SVM for SSL7 with param C=3.0 done...
1NN for SSL7 done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
RFC for SSL7  done...
Ensemble Watersheds for SSL7 with param num_features=20 done...
Ensemble Watersheds for SSL7 with param num_features=30 done...
Ensemble Watersheds for SSL7 with param num_features=50 done...
Watershed for SSL1 done...
priority queue is empty
IFT-SUM for SSL1 done...
RW for SSL1 with param beta=1.0 done...
RW for SSL1 with param beta=3.0 done...
RW for SSL1 with param beta=5.0 done...
PW for SSL1 with param beta=1.0 k=1 

ValueError: shape mismatch: value array of shape (568,) could not be broadcast to indexing result of shape (567,)

In [81]:
import numpy as np
import pandas as pd
from tabulate import tabulate

list_methods = [ "IFT-SUM", "RW", "PW", "SVM", "1NN", "RFC", "watershed", "Ensemblewatershed"]
fnames_dict ={"IFT-SUM":"./results/results_IFT_SUM.csv", 
              "RW":"./results/results_RW.csv", 
              "PW":"./results/results_PW.csv", 
              "SVM":"./results/results_SVM.csv", 
              "1NN":"./results/results_1NN.csv", 
              "RFC":"./results/results_RFC.csv", 
              "watershed": "./results/results_watershed_supervised.csv", 
              "Ensemblewatershed":"./results/results_EnsembleWatersheds.csv"
}

acc_summary = {}
for method in list_methods:
        for dataset in ['SSL1', 'SSL2', 'SSL3','SSL4', 'SSL5','SSL6', 'SSL7']:
            acc_summary[method+dataset] = []


for method in list_methods:
    data = pd.read_csv(fnames_dict[method])
    data = data.fillna(0)
    for dataset in ['SSL1', 'SSL2', 'SSL3','SSL4', 'SSL5','SSL6', 'SSL7']:
        acc_mean, acc_stdev = [], []
        for param in np.unique(data['paramters']):
            ind = np.logical_and.reduce((data['method']==method, data['dataset']==dataset, data['paramters']==param))
            acc_mean.append(np.mean(data[ind]['accuracy']))
            acc_stdev.append(np.std(data[ind]['accuracy']))
        indselect = np.argmax(acc_mean)    
        acc_summary[method+dataset].append((acc_mean[indselect],acc_stdev[indselect]))

l = ["Method Digit1 USPS COIL(binary) BCI g241c COIL g241n".split()]
for method in list_methods:
    tmp = [method]
    for dataset in ['SSL1', 'SSL2', 'SSL3','SSL4', 'SSL5','SSL6', 'SSL7']:
        z = acc_summary[method+dataset]
        tmp.append("{:.2f} + {:.2f}".format(z[0][0]*100, z[0][1]*100))
       
    l.append(tmp)
print(tabulate(l))

-----------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
Method             Digit1        USPS          COIL(binary)  BCI           g241c         COIL          g241n
IFT-SUM            96.75 + 0.64  95.24 + 0.56  95.67 + 0.69  53.75 + 3.45  61.68 + 1.71  89.61 + 0.77  63.89 + 1.27
RW                 98.01 + 0.44  91.44 + 1.09  95.42 + 1.05  52.21 + 2.93  70.37 + 3.70  91.46 + 0.58  75.70 + 4.36
PW                 98.01 + 0.44  95.44 + 0.48  98.42 + 0.35  52.90 + 3.31  70.37 + 3.70  93.33 + 0.74  75.70 + 4.36
SVM                93.63 + 0.71  90.66 + 0.74  56.41 + 0.69  62.17 + 2.61  83.47 + 1.37  22.41 + 1.37  84.18 + 1.02
1NN                96.77 + 0.66  95.30 + 0.54  95.67 + 0.69  53.73 + 3.47  61.66 + 1.71  89.60 + 0.77  63.86 + 1.29
RFC                95.76 + 0.61  88.23 + 0.56  91.78 + 1.25  57.39 + 3.32  74.21 + 1.09  90.57 + 1.33  72.05 + 1.18
watershed          96.53 + 0.91  95.86 + 0.88  99.82 + 0.22  51.65 + 3.04  55.1